In [1]:
import sys
sys.path.append('/home/trukhinmaksim/src')

In [2]:
from pandas import DataFrame as DF, Series
import numpy as np
from pymongo import MongoClient

# single machine setup (mongo is running here localy)
MY_DATABASE_LINK = 'mongodb://10.22.50.212:27020/' #'mongodb://192.168.100.57:27020/'
# multiple mechine setup (mongo is running on another machine)
#MY_DATABASE_LINK = 'mongodb://192.168.43.78:27020/'

class DatabaseConnect:
    DB_LINK = MY_DATABASE_LINK

    class Base:
        client = None
        @classmethod
        def connect(cls, databaseName):
            cls.client = MongoClient(DatabaseConnect.DB_LINK)
            # Access the database
            return cls.client[databaseName]

        @classmethod
        def close(cls):
            if cls.client:
                cls.client.close()
                cls.client = None

        @classmethod
        def getCollection(cls, collectionName):
            return cls.client[collectionName]


    class mini_database(Base):
        @classmethod
        def projects(cls):
            return cls.connect('mini_database')['projects']
        @classmethod
        def users(cls):
            return cls.connect('mini_database')['users']

projectsCollection = DatabaseConnect.mini_database.projects()
print(projectsCollection)


Collection(Database(MongoClient(host=['10.22.50.212:27020'], document_class=dict, tz_aware=False, connect=True), 'mini_database'), 'projects')


In [3]:
from src.utils.CacheAdapter import JSONMultiFileAdapter
from src.utils.DatasetManager import ProjectsDatasetManager

In [4]:
def flatternData(data : dict[str, list]) -> np.array(dict):
    # takes in data in form of dict, where each key is a user id and each value is a list of that user's projects
    # returns just flat list of these projects 
    result = []

    for projectsArray in data.values():
        for project in projectsArray:
            result.append(project)

    return result

def collectCacheStatistics():
    cacheFileName = "cache__02-04-2025__(good)_{0}.json"

    adapter = JSONMultiFileAdapter(cacheFileName)
    USERS_NUMBER_TO_SCAN = 50
    #adapter = JSONAdapter(cacheFileName)

    projectsCount = 0
    manager = ProjectsDatasetManager(USERS_NUMBER_TO_SCAN, cacheAdapter = adapter)

    for i in range(180):
        manager.fromCache()
        #print("Reading:", adapter.collectionName)
        projectsCount += len(flatternData(manager.data))
        manager.clearData()

    return projectsCount

print("Total projects: ", collectCacheStatistics())

Total projects:  57701


In [5]:
2/0
def getProjectsStars() -> list:
    cursor = projectsCollection.find()
    stars = []

    for proj in cursor:
        try:
            stars.append(proj["stars"])
        except KeyError:
            continue

    return np.array(stars)

def percentileThreshold(stars, percentile=50):
    return np.percentile(stars, percentile)

starsArray = getProjectsStars()
starsThreshold = percentileThreshold(starsArray, 90)  # Keep top 10% posts
starsThreshold

ZeroDivisionError: division by zero

In [ ]:
def collectProjectsStatistics():
    global starsThreshold
    stats = Series({
        "Total project" : 0,
        "High quality" : 0,
        "Good" : 0,
        "Sufficient" : 0,
        "Has description" : 0,
        "Has topics" : 0,
        "Avg description length" : 0
    })
    description_lengths = []
    cursor = projectsCollection.find()

    for proj in cursor:
        try:
            desc = proj["description"]
            #print(desc)
        except KeyError:
            continue
        if desc:
            stats["Has description"] += 1
            #print(type(desc))
            description_lengths.append(len(desc))
        
        if len(proj["topics"]):
            stats["Has topics"] += 1

        if desc and (len(proj["topics"]) and proj["language"]) and proj["stars"] >= starsThreshold:
            stats["High quality"] += 1
        
        if desc and (len(proj["topics"]) and proj["language"]):
            stats["Good"] += 1
        
        if desc and (len(proj["topics"]) or proj["language"]):
            stats["Sufficient"] += 1
        stats["Total project"] += 1

    stats["Avg description length"] = sum(description_lengths) / stats["Has description"]

    return stats

In [ ]:
stats = collectProjectsStatistics()

In [ ]:
stats

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
stats.index

In [ ]:
labels = "High quality,Good,Sufficient".split(",")
proportions = [stats[lbl] for lbl in labels]
summa = stats["Sufficient"]
#proportions = proportions / sum(proportions) * 100 # Proportions of each category
proportions[0] = stats["High quality"] / summa * 100
proportions[1] = (stats["Good"] - stats["High quality"]) / summa * 100
proportions[2] = (stats["Sufficient"] - stats["Good"] - stats["High quality"]) / summa * 100
colors = ['red', 'orange', 'pink', 'brown', 'green']
explode = (0,) * len(proportions)  # Explode the 2nd slice

# Creating the pie chart
plt.figure(figsize=(8,4))
plt.pie(proportions, labels=labels, colors=colors, autopct='%1.1f%%', explode=explode, shadow=False, startangle=140)

# Display the chart
plt.axis('equal')  # Ensures the pie is a circle
plt.title("Projects data Distribution by quality")
plt.legend()
plt.show()

In [ ]:
stats

In [ ]:
stats.index
stats["High quality"] / summa * 100

In [ ]:
(6015 - 4258) / summa * 100
proportions

In [ ]:
"""
# calculating projects with camel case
import re

def countCamelCase(camelCase):
    template = "[a-z]+[A-Z]+[a-z]*[A-Z]*[a-z]*[A-Z]*[a-z]*[A-Z]*[a-z]*"
    text = camelCase
    n = 0
    spans = []

    while found := re.search(template, text[n:]):
        spans.append((found.span()[0] + n, found.span()[1] + n))
        n += found.span()[1]

    return len(spans)

def collectProjectsCamelCaseStatistics():
    stats = Series({
        "Total project" : 0,
        "CamelCase projects" : []
    })
    description_lengths = []
    cursor = projectsCollection.find()

    for proj in cursor:
        try:
            desc = proj["description"]
            #print(desc)
        except KeyError:
            continue

        if countCamelCase(desc) > 0:
            stats["CamelCase projects"].append(proj)

        stats["Total project"] += 1

    return stats

stats = collectProjectsCamelCaseStatistics()

print(len(stats["CamelCase projects"]))
print(stats["CamelCase projects"])
"""